## Data 620 - Week 11

By Anjal Hussan, Zhouxin Shi, Chunjie Nan

### Video Presentation



### Assignment


*It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  
For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).*

*For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.*
  

----

## Solution

#### Dataset
For this assignment, I will use the famous Reuters news dataset that comes with the NLTK package. According to the documentaion, "The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called 'training' and 'test'. [1]
  
#### Approach
The analysis is conducted as follows and uses the functionality of `nltk` and `sklearn` libraries and the general approach to training an LDA classifier outlined in [2,3].
  
1) Read and prepare the training documents  
2) Train an LDA model on the training dataset  
3) Apply the model on the test data and evaluate its performance in terms of correct topic recognition  

## 1. Setup and read the data

In [1]:
import numpy as np
from nltk.corpus import reuters
from nltk.stem.porter import PorterStemmer
from nltk import FreqDist
import string



In [2]:
reuters.fileids()[:5]

['test/14826', 'test/14828', 'test/14829', 'test/14832', 'test/14833']

First, we need to inspect which are the 90 topics listed in the documentation and how often they appear in the data

In [3]:
for item in reuters.categories(): print (item,end =" "),

acq alum barley bop carcass castor-oil cocoa coconut coconut-oil coffee copper copra-cake corn cotton cotton-oil cpi cpu crude dfl dlr dmk earn fuel gas gnp gold grain groundnut groundnut-oil heat hog housing income instal-debt interest ipi iron-steel jet jobs l-cattle lead lei lin-oil livestock lumber meal-feed money-fx money-supply naphtha nat-gas nickel nkr nzdlr oat oilseed orange palladium palm-oil palmkernel pet-chem platinum potato propane rand rape-oil rapeseed reserves retail rice rubber rye ship silver sorghum soy-meal soy-oil soybean strategic-metal sugar sun-meal sun-oil sunseed tea tin trade veg-oil wheat wpi yen zinc 

In [4]:
# Count the frequency of the topics

cat_freq = []
for elem in reuters.fileids():
    cat_freq.append(reuters.categories(elem))

topics = [' '.join(item) for item in cat_freq]
topics_freq = FreqDist(topics)

# Order topics by frequency
topics_freq.most_common(10)

[('earn', 3923),
 ('acq', 2292),
 ('crude', 374),
 ('trade', 326),
 ('money-fx', 309),
 ('interest', 272),
 ('interest money-fx', 167),
 ('money-supply', 151),
 ('grain wheat', 150),
 ('ship', 144)]

We can see that by far the most common topics are the inscrutiable terms "earn" and "acq". 
  
The documents tagged as "earn" are related to the earnings of corporate shareholders:

In [5]:
for item in reuters.words(categories='earn')[:100] : print (item,end =" "), 

AMATIL PROPOSES TWO - FOR - FIVE BONUS SHARE ISSUE Amatil Ltd & lt ; AMAA . S > said it proposes to make a two - for - five bonus issue out of its revaluation reserve to shareholders registered May 26 . Shareholders will be asked to approve the issue and an increase in authorised capital to 175 mln shares from 125 mln at a general meeting on May 1 , it said in a statement . The new shares will rank for dividends declared after October 31 . Amatil , in which B . A . T . 

In [6]:
for item in reuters.words(categories='earn')[500:600] : print (item,end =" "), 

while the profit from U . K . Operations rose 30 . 7 pct to 24 . 7 mln , and Europe , 42 . 9 pct to 11 . 0 mln . CITIBANK NORWAY UNIT LOSES SIX MLN CROWNS IN 1986 Citibank A / S & lt ; CCI . N >, the Norwegian subsidiary of the U . S .- based bank , said it made a net loss of just over six mln crowns in 1986 -- although foreign bankers said they expect it to show 1987 profits after two lean years . Citibank ' s Oslo 

The documents tagged as "acq" are related to corporate mergers and acquisitions :

In [7]:
for item in reuters.words(categories='acq')[:100] : print (item,end =" "), 

SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER Sumitomo Bank Ltd & lt ; SUMI . T > is certain to lose its status as Japan ' s most profitable bank as a result of its merger with the Heiwa Sogo Bank , financial analysts said . Osaka - based Sumitomo , with desposits of around 23 . 9 trillion yen , merged with Heiwa Sogo , a small , struggling bank with an estimated 1 . 29 billion dlrs in unrecoverable loans , in October . But despite the link - up , Sumitomo President Koh Komatsu told Reuters 

In [8]:
for item in reuters.words(categories='acq')[1000:1100] : print (item,end =" "), 

, to be repaid by the mining company in gold . Atlas said the two sides were also discussing equity infusion into Atlas and the creation of a development fund for further exploration and development of the company ' s gold properties in the central province of Masbate . Wilson Banks , general manager of & lt ; Bond Corp International Ltd > in Hong Kong , told Reuters the Atlas statement on the negotiations was " reasonably accurate ." Banks said Bond Corp was seriously considering several investments in the Philippines but did not give details . In its 

For further modeling, these two top topics will be used exclusively, meaning that the topic model should be able to infer if a document relates to "earn", "acq", or both topics.

In [9]:
# Create filtered training and test datasets
reuters_filt_ids = reuters.fileids(categories=["earn","acq"])

We can see that the topics of the text are "crude"(oil) and "ship". But in order to model the topics, we need to convert each of the training texts into a bag of stemmed words, exclude punctuation and numbers, and exclude the "stopwords" occurring across all topics. 

## 2. Clean the input and train the LDA classifier

In [10]:
train_fileids = [item for item in reuters_filt_ids if 'train' in item]
test_fileids = [item for item in reuters_filt_ids if 'test' in item]

train_docs = [reuters.words(item) for item in train_fileids]
train_docs = [' '.join(item) for item in train_docs]

test_docs = [reuters.words(item) for item in test_fileids]
test_docs = [' '.join(item) for item in test_docs]

In [11]:
print ("Training documents: {}, test documents: {}".format(len(train_fileids), len(test_fileids)))

Training documents: 4511, test documents: 1803


First, we build a vocabulary of the thousand top most frequent terms appearing in all of the training documents minus the English language stop words. A vocabulary of a thousand terms should be sufficient to have enough variation in order to describe each of the two topics using different terms.

In [12]:
import sys
!{sys.executable} -m pip install sklearn

You should consider upgrading via the 'C:\Users\jshi3\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vocab_size = 1000

# words occurring in only two documents or in at least 90% of the documents are removed.

tf_vectorizer = CountVectorizer(max_df=0.9, min_df=5, max_features=vocab_size, 
                                analyzer = "word", token_pattern = r'\b[a-z]\w+\b', 
                                stop_words='english', strip_accents="unicode")

tf = tf_vectorizer.fit_transform(train_docs)
tf_feature_names = tf_vectorizer.get_feature_names()

C:\Users\jshi3\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Now we can apply the Latent Dirichlet Allocation model to find the words most characteristic of each of the 90 topics from the vocabulary we have built.

In [14]:
# Run LDA
from sklearn.decomposition import LatentDirichletAllocation

random_seed = 123


lda = LatentDirichletAllocation(max_iter=5, 
                                learning_method='online', 
                                learning_offset=50.,random_state=random_seed).fit(tf)


In [15]:
def print_top_words(model, feature_names, n_top_words):
    """
    # Author: Olivier Grisel <olivier.grisel@ensta.org>
    #         Lars Buitinck
    #         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
    # License: BSD 3 clause
    Source: http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html
    """
    
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
no_top_words = 10
print_top_words(lda, tf_feature_names, 20)

Topic #0:
usair twa piedmont said air purolator southern department interstate pacific hutton airlines order merger courier application boston offer fleet group
Topic #1:
cts april record dividend lt div vs pay qtly prior march sets quarterly payable payout corp declared split regular june
Topic #2:
shares said stock pct common lt company share stake group outstanding mln dlrs board split shareholders exchange securities corp cyclops
Topic #3:
vs mln cts net loss shr dlrs profit revs qtr year lt note oper avg shrs sales includes share corp
Topic #4:
offer said dlrs bid share tender lt company gencorp takeover says management dlr taft corp shares shareholders brown group today
Topic #5:
said lt company mln sale bank unit pct corp sell dlrs banks new stake business group companies international financial subsidiary
Topic #6:
american said lt corp dlrs general stores international mln unit western acquired siemens allied allegheny motors chrysler new supermarkets harper
Topic #7:
billion 

The inspection of the topic top words shows that "Topic #0" listing the terms "profit, loss, billion" corresponds to the input topic "earn", and "Topic #1" (terms like "said", "company", "offer", "common") corresponds to "acq".

## 3. Test and tune the topic model

Now we'll use the trained LDA classifier to evaluate the documents in the test set and get the topic probability per document using the normalization approach described in [6].  
After this step, the accuracy of the classification can be measured vs. the original topic labels

In [16]:
# predict topics for test data
# unnormalized doc-topic distribution
tf_test = tf_vectorizer.transform(test_docs)
doc_topic_dist_unnormalized = np.matrix(lda.transform(tf_test))

# normalize the distribution 
doc_topic_dist = doc_topic_dist_unnormalized / doc_topic_dist_unnormalized.sum(axis=1)

doc_topic_dist

matrix([[5.34845884e-04, 5.34936004e-04, 3.22394864e-02, ...,
         7.56525167e-02, 1.01324967e-01, 9.34002783e-02],
        [9.26036082e-04, 9.26006389e-04, 9.26078264e-04, ...,
         6.95186654e-02, 4.04241431e-01, 3.49386636e-02],
        [2.17413261e-03, 5.88536797e-02, 5.45362201e-01, ...,
         2.17441479e-03, 2.65105924e-01, 2.17441299e-03],
        ...,
        [3.22603153e-03, 3.22621249e-03, 3.22639229e-03, ...,
         4.33493450e-01, 3.22692586e-03, 3.22636562e-03],
        [9.09202202e-03, 9.09093924e-03, 9.09181485e-03, ...,
         9.09102261e-03, 9.09156662e-03, 1.13317235e-01],
        [1.28247653e-03, 1.28213882e-03, 3.75531558e-01, ...,
         1.28232373e-03, 1.28249501e-03, 1.28234208e-03]])

As we can see in the examples above, the assignment to the topic#0 (first column) or topic#1 (second column) is fairly exclusive.

In [17]:
# Get the most probable topic per test document and join to the document id
topic_guess = [item[0] for item in np.array(doc_topic_dist.argmax(axis=1))]

topic_guess_label = []
for item in topic_guess:
    if item == 1:
        topic_guess_label.append("acq")
    else:
        topic_guess_label.append("earn")

In [18]:
test_true = [reuters.categories(item) for item in test_fileids]

In [19]:
test_true_label = []

for item in test_true:
    if "acq" in item:
        test_true_label.append("acq")
    elif "earn" in item:
        test_true_label.append("earn")

#print zip(test_true[:15], test_true_label[:15])

In [20]:
# Compare the assignments
# Source: https://stackoverflow.com/a/29877565/8066374

import pandas as pd
y_actu = pd.Series(test_true_label, name='Actual')
y_pred = pd.Series(topic_guess_label, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,acq,earn,All
Actual,,,
acq,0,719,719
earn,75,1009,1084
All,75,1728,1803


The confusion matrix shows the following classification performance:

In [21]:
accuracy = (716.0 + 929)/1803
print ("Topic classification accuracy for the two topics: {}%".format(round(accuracy*100,1)))

Topic classification accuracy for the two topics: 91.2%


We can also see that the LDA classifier tends to assign documents labeled as "earn" to the "acq" topic.
If we had multiple dev_test sets, we could tune the sensitivity of the classifier by adjusting the probability threshold for the "acq" topic so as to reduce the faulty class assignments.
  
The example on the test set below shows how this could possibly work:  
1) Calculate the mean probabilities for "acq" in the error cases and set it as a threshold probability for the class assignment  
2) Re-evaluate the data using the threshold value from (1) instead of a simple argmax class probability

In [22]:
# Get the distribution of class probabilites for the faulty assignments

import numpy as np

topic_guess2 = [item[0] for item in np.array(doc_topic_dist)]
topic_guess3 = [item[1] for item in np.array(doc_topic_dist)]

topic = pd.DataFrame(topic_guess2,columns=["0_earn"])
topic2 =  pd.DataFrame(topic_guess2,columns=["1_acq"])

df = pd.concat([y_actu,y_pred,topic,topic2],axis=1)
errors = df[df.Actual != df.Predicted]
errors[errors.Predicted == "acq"]["1_acq"].describe()

count    75.000000
mean      0.006379
std       0.001772
min       0.001351
25%       0.005556
50%       0.006667
75%       0.007419
max       0.014286
Name: 1_acq, dtype: float64

In [23]:
df.head()

,Actual,Predicted,0_earn,1_acq
0,acq,earn,0.000535,0.000535
1,acq,earn,0.000926,0.000926
2,earn,earn,0.002174,0.002174
3,earn,earn,0.001449,0.001449
4,acq,earn,0.002222,0.002222


In [24]:
# Re-evaluate class assignment using the mean probability in the error cases as a new threshold for "acq"
df["new_Pred"] = "earn"
df.loc[df["1_acq"] >= 0.671, "new_Pred"] = "acq"

df_confusion_new = pd.crosstab(df["Actual"],df["new_Pred"], rownames=['Actual'], 
                               colnames=['New_Predicted'], margins=True)
df_confusion_new

New_Predicted,acq,earn,All
Actual,,,
acq,2,717,719
earn,0,1084,1084
All,2,1801,1803


In [25]:
accuracy_new = (703.0 + 1018)/1803
print ("Topic classification accuracy for the two topics after tuning: {}%".format(round(accuracy_new*100,1)))

Topic classification accuracy for the two topics after tuning: 95.5%


We get a higher overall accuracy as a result (but we cannot test if this model is now overfitting the data as we've run out of test sets).

----
  
### Reference
1) http://www.nltk.org/book/ch02.html  
2) https://medium.com/@aneesha/topic-modeling-with-scikit-learn-e80d33668730  
3) http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html  
4) http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html  
5) http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html  
6) https://stackoverflow.com/questions/40597075/python-sklearn-latent-dirichlet-allocation-transform-v-fittransform  
7) https://stackoverflow.com/a/29877565/8066374 